In [88]:
import pandas as pd
import numpy as np
import librosa
from librosa import feature
from glob import glob
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
min_max_scaler = preprocessing.MinMaxScaler()


In [12]:
rootdir = '/Users/abhishekvaidyanathan/Desktop/NNDL-project/audio-files'
audio_files = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        audio_files.append(os.path.join(subdir, file))
audio_files = audio_files[1:]

In [16]:
audio_files[1][-33:-25]

'Actor_16'

In [22]:
dict_actors = {}
for audio_file in audio_files:
    try :
        if (len(dict_actors[audio_file[-33:-25]])>0):
            dict_actors[audio_file[-33:-25]].append(audio_file)
    except:
        dict_actors[audio_file[-33:-25]] = []
        dict_actors[audio_file[-33:-25]].append(audio_file)

In [23]:
data = pd.DataFrame(columns = ['Actor','Modality','Vocal_channel','Emotion','Emotional_intensity','Statement','Repetion','Gender','Audio_file'])

In [24]:
def get_gender(value):
    if(int(value)%2==0):
        return 1
    else :
        return 0

for keys in dict_actors:
    for files in dict_actors[keys]:
        data.loc[len(data)] = [keys,int(files[-24:-22]),int(files[-21:-19]),int(files[-18:-16]),int(files[-15:-13]),int(files[-12:-10]),int(files[-9:-7]),get_gender(files[-6:-4]),files]

In [25]:
data.head()

,Actor,Modality,Vocal_channel,Emotion,Emotional_intensity,Statement,Repetion,Gender,Audio_file
0,Actor_16,3,1,5,1,2,1,1,/Users/abhishekvaidyanathan/Desktop/NNDL-proje...
1,Actor_16,3,1,6,1,2,2,1,/Users/abhishekvaidyanathan/Desktop/NNDL-proje...
2,Actor_16,3,1,6,2,1,2,1,/Users/abhishekvaidyanathan/Desktop/NNDL-proje...
3,Actor_16,3,1,5,2,1,1,1,/Users/abhishekvaidyanathan/Desktop/NNDL-proje...
4,Actor_16,3,1,7,1,1,1,1,/Users/abhishekvaidyanathan/Desktop/NNDL-proje...


## can change the below code cell to include different set of features. 

#### right now uses mean values, can change to using raw values for each of the features. 

In [26]:
fn_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff
]
 
fn_list_ii = [
 feature.rms,
 feature.zero_crossing_rate
]
def get_feature_vector(y,sr): 
   feat_vect_i = [ np.mean(funct(y,sr)) for funct in fn_list_i]
   feat_vect_ii = [ np.mean(funct(y)) for funct in fn_list_ii] 
   feature_vector = feat_vect_i + feat_vect_ii 
   return feature_vector

In [27]:
audio_features = []
for i in range(data.shape[0]):
   y , sr = librosa.load(data.iloc[i]['Audio_file'],sr=None)
   feature_vector = get_feature_vector(y, sr)
   audio_features.append(feature_vector) 

In [31]:
data_features = data.copy()

In [34]:
data_features = data_features.drop("Audio_file",axis=1)
data_features.head()

,Actor,Modality,Vocal_channel,Emotion,Emotional_intensity,Statement,Repetion,Gender
0,Actor_16,3,1,5,1,2,1,1
1,Actor_16,3,1,6,1,2,2,1
2,Actor_16,3,1,6,2,1,2,1
3,Actor_16,3,1,5,2,1,1,1
4,Actor_16,3,1,7,1,1,1,1


In [80]:
data_features['librosa'] = audio_features

In [81]:
features = pd.DataFrame(data_features['librosa'].values.tolist())

In [82]:
statement1 = 'Kids are talking by the door'
statement2 = 'Dogs are sitting by the door'

In [83]:
features.head()

,0,1,2,3,4,5
0,0.417426,5197.643881,4276.516635,9398.629611,0.011657,0.074035
1,0.349234,4250.653570,3677.994703,7682.112873,0.008692,0.070726
2,0.407229,5423.119278,4838.148261,10376.602564,0.038229,0.117620
3,0.389465,4646.002917,4298.393109,9022.055127,0.023401,0.070798
4,0.368242,4275.794918,3958.305217,8093.021716,0.004161,0.062740


In [84]:
labels = data['Emotion']

In [85]:
features_values = features.values

In [86]:
features_scaled_values = min_max_scaler.fit_transform(features_values)
features_normalised = pd.DataFrame(features_scaled_values)

In [87]:
features_normalised.head()

,0,1,2,3,4,5
0,0.253042,0.513457,0.421372,0.456914,0.128488,0.328955
1,0.086784,0.325949,0.255784,0.278685,0.094858,0.305460
2,0.228181,0.558102,0.576754,0.558458,0.429833,0.638509
3,0.184870,0.404229,0.427425,0.417813,0.261668,0.305969
4,0.133126,0.330927,0.333336,0.321350,0.043477,0.248738


In [89]:
X_train, X_test, y_train, y_test = train_test_split(features_normalised, labels, test_size=0.33, random_state=42)